<a href="https://colab.research.google.com/github/kidusalamrewsqralex-droid/my_model/blob/main/Smart_Plant_Disease_Detection_System(SPDDS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q tensorflow tensorflow-datasets

In [4]:
!pip install tensorflow.keras

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset,info=tfds.load('plant_village',with_info=True,as_supervised=True)
teain_ds_full=dataset['train']
print('Number of classes: ',info.features['label'].num_classes)
print('Class names: ',info.features['label'].names)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/plant_village/incomplete.JMWV79_1.0.2/plant_village-train.tfrecord*...:   …

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.
Number of classes:  38
Class names:  ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy', 'Cherry___Powdery_mildew', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_bligh

In [3]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

train_ds_full = dataset['train'].map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

total_examples = info.splits["train"].num_examples
train_size = int(0.8 * total_examples)
val_size = total_examples - train_size

train_ds = train_ds_full.take(train_size).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = train_ds_full.skip(train_size).take(val_size).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),

    Dense(38, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1206s 887ms/step - accuracy: 0.8135 - loss: 0.6135 - val_accuracy: 0.8615 - val_loss: 0.4458
Epoch 2/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1213s 892ms/step - accuracy: 0.9017 - loss: 0.3062 - val_accuracy: 0.8638 - val_loss: 0.4451
Epoch 3/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1228s 897ms/step - accuracy: 0.9347 - loss: 0.1959 - val_accuracy: 0.8883 - val_loss: 0.3775
Epoch 4/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1090s 802ms/step - accuracy: 0.9567 - loss: 0.1281 - val_accuracy: 0.8664 - val_loss: 0.4871
Epoch 5/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1113s 819ms/step - accuracy: 0.9669 - loss: 0.0993 - val_accuracy: 0.9005 - val_loss: 0.3939
Epoch 6/20
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1092s 803ms/step - accuracy: 0.9778 - loss: 0.0683 - val_accuracy: 0.8865 - val_loss: 0.4663


In [10]:
model.save("SPDDS.keras")

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
model.save('/content/drive/MyDrive/my_model.keras')

In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
model = load_model('/content/drive/MyDrive/my_model.keras')
img = image.load_img('thumb.jpg', target_size=(128, 128),color_mode='rgb')
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)
predicted_class=np.argmax(prediction,axis=1)[0]
class_names=info.features['label'].names
print('prediction: ',class_names[predicted_class])
print("Prediction probabilities:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
prediction:  Apple___Apple_scab
Prediction probabilities: [[9.9371535e-01 1.8196980e-07 1.5017719e-12 3.9703473e-03 5.1309118e-12
  4.7156419e-12 3.4193735e-11 2.4655970e-05 1.3261789e-12 6.6917149e-12
  9.7061343e-07 1.9866608e-09 1.1667213e-10 9.8081673e-12 1.1307223e-09
  1.3089012e-10 5.0976546e-04 5.8958873e-11 2.7914542e-08 2.8173398e-14
  1.1652433e-05 6.7596888e-09 5.1173207e-04 4.4710608e-13 1.2268320e-10
  1.2541467e-03 2.5334936e-11 4.8117312e-07 1.1596219e-10 8.8753197e-08
  6.8693735e-09 4.2034034e-07 3.1075952e-08 1.1317008e-07 1.7533339e-11
  1.0398152e-07 7.0387022e-17 7.7360561e-15]]
